In [2]:
import math
import logging
FORMAT = '[%(name)s:%(levelname)s]  %(message)s'
logging.basicConfig(level=logging.DEBUG, format=FORMAT)
logger = logging.getLogger('dbg')

def dprint(s):
    logger.debug(s)

def iprint(s):
    logger.info(s)

logger.setLevel(logging.INFO)

In [9]:
## Graphs Class for demonstration of algorithms

## Directed Edge Object
class Edge():
    def __init__(self, a, b, weight) -> None:
        self.a = a
        self.b = b
        self.w = weight
    
    def __repr__(self) -> str:
        return f"{self.a}<->{self.b} [{self.w}]"

## Vertex Object
class Vertex():
    def __init__(self, v) -> None:
        self.v = v

        self.color = "white"
        self.distance = float('inf')
        self.pi = None
        self.d = 0
        self.f = 0

    ## configure type comparisons
    def __contains__(self, item):
        if isinstance(item, Vertex):
            return item.v == self.v
        return item == self.v
    def __eq__(self, item):
        if isinstance(item, Vertex):
            return item.v == self.v
        return item == self.v
    
    def __repr__(self) -> str:
        if self.pi is not None:
            return f"{self.v} parent[{self.pi.v}] C:[{self.distance}] d[{self.d}] f[{self.f}] [{self.color}]"
        else:
            return f"{self.v}  C:[{self.distance}] d[{self.d}] f[{self.f}] [{self.color}]"

class Graph():
    ## Internal Adj matrix
    
    ## Build from lists by default
    # V: List of Vertex Elements
    # E: List of Edge Elements
    def __init__(self, V: list, u: list, v: list, w: list) -> None:
        self.V = []
        for q in V:
            self.V.append(Vertex(q))
        self.E = []

        # if we have edge data
        if u is not None:
            for i in range(len(u)):
                self.E.append(Edge(u[i], v[i], w[i]))
            # construct adjacency matrix/dict
            self.build_dict()

    def load_edge_from_tuple(self, E: list) -> None:
        self.E = []
        for i in range(len(E)):
            self.E.append(Edge(E[i][0], E[i][1], E[i][2]))
        # construct adjacency matrix/dict
        self.build_dict()

    def __repr__(self) -> str:
        return str(self.Aj)

    def build_array(self):
        self.Aj = []
        ## Create the array
        self.Aj = [ [ 0 for _ in self.V] for _ in self.V]
        ## fill it in
        for e in self.E:
            u_i = self.V.index(e.a)
            v_i = self.V.index(e.b)
            self.Aj[u_i][v_i] = e.w

    def build_dict(self):
        self.Aj = dict()
        # allocate the primary dict
        for vert in self.V:
            self.Aj[vert.v] = dict()
        ## fill it in
        for edge in self.E:
            self.Aj[edge.a][edge.b] = edge.w

    def get_vertex(self, a) -> Vertex:
        return self.V[self.V.index(a)]

    def to_undirected(self):
        for i in range(len(self.E)):
            self.E.append(Edge(self.E[i].b, self.E[i].a, self.E[i].w))
        # construct adjacency matrix/dict
        self.build_dict()



_V = ["s","a","b","c","d","e","f","g"]
_u = ["s","s","a","b","d","d","e","e","c"]
_v = ["a","b","d","e","e","g","d","c","f"]
_w = [1 for _ in _v]

G = Graph(_V, _u, _v, _w)
print(G)

{'s': {'a': 1, 'b': 1}, 'a': {'d': 1}, 'b': {'e': 1}, 'c': {'f': 1}, 'd': {'e': 1, 'g': 1}, 'e': {'d': 1, 'c': 1}, 'f': {}, 'g': {}}


## Graphs - Basics

A graph is a mathematical object containing a set of *vertices* or corners $V$, and a set of *edges* connecting these vertices $E$. 

**Common representations:**
- *Adjacency List* - an array $A_j$ of $|V|$ unique lists, where each index of $A_j$ corresponds to a vertex $u \in V$, and the indexed list contains all vertices $v$ where there is an edge from $u$ to $v$ or equivalently  $(u,v) \in E$

Memory: **$\Theta(|V| + |E|)$**

E.g. for $|V|$ = 5, $V$ = $[1, 2, 3, 4, 5]$, a encyclic circular graphs could be described as:

| $u$ = 1   | $u$ = 2   | $u$ = 3   | $u$ = 4   | $u$ = 5   |
| --        | --        | --        | --        | --        |          
| 2         | 3         | 4         | 5         | 1         |

- *Adjacency Matrix* - a $|V|\times|V|$ binary matrix where 1 corresponds to $E$, and 0 not.

Memory: **$\Theta(|V|^2)$**

E.g. for $|V|$ = 5, $V$ = $[1, 2, 3, 4, 5]$, a encyclic circular graphs could be described as:

| $u$ = 1   | $u$ = 2   | $u$ = 3   | $u$ = 4   | $u$ = 5   |
| --        | --        | --        | --        | --        |          
| ~         | 0         | 0         | 0         | 1         |
| 1         | ~         | 0         | 0         | 0         |
| 0         | 1         | ~         | 0         | 0         |
| 0         | 0         | 1         | ~         | 0         |
| 0         | 0         | 0         | 1         | ~         |


**Types:**
* Directed
* Undirected
* Weighted
* Unweighted

**Algorithms:**
* **Graph Search & Traversal**
    * Breadth-First Search (BFS)
    * Depth-First Search (DFS)
* **Minimum Weight Spanning Trees**
    * Kruskal's Al
    * Primm's Al
* **Shortest Path**
    * Bellman-Ford 
    * Dijkstra
    * Floyd-Warshall
* **Maximum Flow**
    * Ford-Fulkerson


## Search & Traversal

### BFS

Expands the frontier between discovered and undiscovered vertices uniformly across the breadth. BFS finds all vertices at a distance $k$ from source $s$, before discovering any that are at a distance $k+1$.

Given a graph $G=(V, E)$, and a **source** vertex $S$:
* Discover every vertex that is reachable from $S$
* Discover distance $d(v)$ from $S$ to each reachable vertext $v$
* Create a BFS tree, starting at $S$, that contains all reachable vertices

**Implementation:**
* Classify vertices into: 
    * undiscovered (white)
    * a frontier between discovered and undiscovered vertices (gray)
    *discovered vertices (black).
* Starting from the first vertex on frontier, append all undiscovered neighboring nodes 
* Proceed until all are discovered


In [10]:
# BFS Implementation

_V = ["s","a","b","c","d","e","f","g"]
_u = ["s","s","a","b","d","d","e","e","c"]
_v = ["a","b","d","e","e","g","d","c","f"]
_w = [1 for _ in _v]

G = Graph(_V, _u, _v, _w)

def init_bfs(G: Graph, s: Vertex):
    for v in G.V:
        v.color = "white"
        v.distance = float('inf')
        v.pi = None
        
    s.color = "gray"
    s.distance = 0

def bfs(G: Graph, s: Vertex):
    init_bfs(G, s)
    Q = []; Q.append(s)
    while len(Q):
        u = Q.pop(0)
        # loop through th edges belonging to u
        for v in G.Aj[u.v]:
            w = G.Aj[u.v][v]
            vv = G.get_vertex(v)
            if (vv.color == "white"):
                vv.color = "gray"
                vv.distance = u.distance + 1
                vv.pi = u
                Q.append(vv)
            u.color = "black"

bfs(G, G.get_vertex("s"))

for v in G.V:
    print(v)


s  C:[0] d[0] f[0] [black]
a parent[s] C:[1] d[0] f[0] [black]
b parent[s] C:[1] d[0] f[0] [black]
c parent[e] C:[3] d[0] f[0] [black]
d parent[a] C:[2] d[0] f[0] [black]
e parent[b] C:[2] d[0] f[0] [black]
f parent[c] C:[4] d[0] f[0] [gray]
g parent[d] C:[3] d[0] f[0] [gray]


### DFS

Choose a first undiscovered vertex and explore neighbors recursively until no unexplored neighbors are left. For each vertex $v$, once all out going edges are discovered, the algorithm must backtrack to vertex from which $v$ was discovered.

Undiscovered vertices are coloured `white`, those with fully explored neighbors `black`, and the rest `gray`.

Given $G$, compute a predecessor subgraph $G_\pi$.
* The graph comprises several depth-first-search trees
* Structural information is obtained via start and finish times of each node

In [11]:
_V = ["s","a","b","c","d","e","f","g"]
ET= [("s", "b", 1), \
    ("s", "c", 1), \
    ("a", "s", 1), \
    ("a", "b", 1), \
    ("b", "c", 1), \
    ("b", "d", 1), \
    ("c", "s", 1), \
    ("d", "a", 1), \
    ("e", "b", 1), \
    ("f", "c", 1), \
    ("f", "e", 1), \
    ("f", "g", 1), \
    ("g", "d", 1), \
    ("g", "e", 1), \
    ("g", "f", 1)]
G = Graph(_V, None, None, None)
G.load_edge_from_tuple(ET)

# DFS Implementation
time = 0
def DFS_visit(G: Graph, u: Vertex):
    global time
    time += 1
    u.d = time
    u.color = "gray"
    for v in G.Aj[u.v]:
        w = G.Aj[u.v][v]
        vv = G.get_vertex(v)
        if (vv.color == "white"):
            vv.pi = u
            DFS_visit(G, vv)
    u.color = "black"
    time += 1
    u.f = time

def DFS(G: Graph):
    global time
    time = 0
    for v in G.V:
        v.color = "white"
        v.d = 0
        v.f = 0
        v.pi = None
    
    for u in G.V:
        if u.color == "white":
            DFS_visit(G, u)
    return time


DFS(G)
for v in G.V:
    print(v)
    

s  C:[inf] d[1] f[10] [black]
a parent[d] C:[inf] d[6] f[7] [black]
b parent[s] C:[inf] d[2] f[9] [black]
c parent[b] C:[inf] d[3] f[4] [black]
d parent[b] C:[inf] d[5] f[8] [black]
e  C:[inf] d[11] f[12] [black]
f  C:[inf] d[13] f[16] [black]
g parent[f] C:[inf] d[14] f[15] [black]


### Topological Sort

A Topological sort of a directed acycled graph (DAG) is a linear ordering of its vertices such that an edge $(u, v)$ requires $u$ appear before $v$ in the ordering.

If the graph contains a cycle this is of course impossible.

In [12]:
def toposort(G: Graph):
    t = DFS(G)
    A = [-1 for i in range(t+1)]
    for v in G.V:
        A[v.f] = v
    A = list(filter(lambda a: a != -1, A))
    A.reverse()
    return A

_V = ["s","a","b","c","d","e","f","g"]
ET= [("s", "b", 1), \
    ("s", "c", 1), \
    ("b", "c", 1), \
    ("b", "d", 1), \
    ("d", "a", 1), \
    ("e", "b", 1), \
    ("f", "c", 1), \
    ("f", "e", 1), \
    ("f", "g", 1), \
    ("g", "d", 1), \
    ("g", "e", 1)]

G = Graph(_V, None, None, None)
G.load_edge_from_tuple(ET)

A = toposort(G)
for v in A:
    print(v)

f  C:[inf] d[13] f[16] [black]
g parent[f] C:[inf] d[14] f[15] [black]
e  C:[inf] d[11] f[12] [black]
s  C:[inf] d[1] f[10] [black]
b parent[s] C:[inf] d[2] f[9] [black]
d parent[b] C:[inf] d[5] f[8] [black]
a parent[d] C:[inf] d[6] f[7] [black]
c parent[b] C:[inf] d[3] f[4] [black]


### Strongly Connected Components

A strongly connected set of a directed graph is a maximal set of vertices such that for every set of vertices, there is a path from one to the other and the other to one.

* Call `DFS` to compute the finishing times `f` for every node
* Compute the graph's transpose $G^T$
* Call DFS $G^T$ but in the main loop consider in order of decreasing `f`
* Output the vertices in the depth-first forest as separately strongly connected components.

### Minimum Weight Spanning Trees

A minimum weight spanning trr of an undirected graph G is an acyclic subset of edges that connects all vertices and whose total weight is minimized. MST's can be constructed using a greedy strategy maintaining the proper invariant:

**Prior to each iteration, A is a subset of a MST**

At each step, a safe edge is added that respects the invariant.

Generic MST:
* $A$ is the empty set
* while $A$ does not form an MST
* find an edge that is safe and add it to $A$

#### Safe Edges
* A cut of an undirected graph is a partition of its vertices
* An edge may cross a cut if it posses a vertex either side
* An edge is a light edge crossing a cut if its weight is the minimum of those that cross

Hence any *light edge* is safe to be appended to $A$


#### Kruskal's Algorithm for MST

Create a growing collection of multiple forests. Find a safe edge to add to the forest by finding all edges that connect any two trees and choosing the light edge. Automatically ends when all edges are accessible in a single forest.

Runtime is $O(|E| \log |V|)$ for an $O( n \log n)$ implementation of the set handling routines.

In [16]:
_V = ["s","a","b","c","d","e","f","g"]
ET= [("s", "a", 2), \
    ("s", "c", 1), \
    ("a", "d", 4), \
    ("a", "b", 2), \
    ("b", "c", 2), \
    ("b", "f", 6), \
    ("b", "e", 3), \
    ("e", "f", 1), \
    ("f", "g", 2), \
    ("d", "e", 1), \
    ("e", "f", 1)]

G = Graph(_V, None, None, None)
G.load_edge_from_tuple(ET)
G.to_undirected()

def find_tree(treelist: list, x: str) -> int:
    # get the index that contains the node x
    for i, l in enumerate(treelist):
        if x in l:
            return i
    raise Exception

def MST_kruskal(G):
    A = []
    Vset = []

    # make a deep copy of the vertex list where each is a tree (list of vertices)
    for v in G.V:
        Vset.append([Vertex(v.v)])

    # sort the edges into ascending order by weight
    local_E = sorted(G.E, key=lambda x: x.w, reverse=False)
    
    for e in local_E:
        # if the nodes of the edge are in two different trees
        if find_tree(Vset, e.a) != find_tree(Vset, e.b):
            # edge is safe, add it to the forest
            A.append(e)
            # combine the two trees
            newset = Vset.pop( find_tree(Vset, e.a))
            newset += Vset.pop( find_tree(Vset, e.b))
            Vset.append(newset)
    return A

A = MST_kruskal(G)
print(A)

[[s  C:[inf] d[0] f[0] [white], c  C:[inf] d[0] f[0] [white], a  C:[inf] d[0] f[0] [white], b  C:[inf] d[0] f[0] [white], d  C:[inf] d[0] f[0] [white], e  C:[inf] d[0] f[0] [white], f  C:[inf] d[0] f[0] [white], g  C:[inf] d[0] f[0] [white]]]
[s<->c [1], e<->f [1], d<->e [1], s<->a [2], a<->b [2], f<->g [2], b<->e [3]]


#### Prim's Algorithm

* Start from an arbitrary root $r$
* Edges in the set $A$ always form a single tree
* Each step adds a light edge to $A$ that connects $A$ to an isolated vertex
* Each vertex stores the minimum weight of any edge connected from the vertex to another in the tree
* Vertices also store their parent
* $Q$ is a min-priority queue, used to find the light edge based on the vertex edge min weight above

Runtime is $O(|E| \log |V|)$ if a binary min heap is used.

In [ ]:
_V = ["s","a","b","c","d","e","f","g"]
ET= [("s", "a", 2), \
    ("s", "c", 1), \
    ("a", "d", 4), \
    ("a", "b", 2), \
    ("b", "c", 2), \
    ("b", "f", 6), \
    ("b", "e", 3), \
    ("e", "f", 1), \
    ("f", "g", 2), \
    ("d", "e", 1), \
    ("e", "f", 1)]

G = Graph(_V, None, None, None)
G.load_edge_from_tuple(ET)
G.to_undirected()

def MST_prim(G: Graph, r: Vertex):
    # reuse some params from the searches
    for v in G.V:
        v.distance = float('inf')
        v.pi = None
    r.distance = 0

    # deep copy
    Q = []
    for v in G.V:
        Q.append(Vertex(v.v))
    
    # Calc min queue Q
    # etc....
    

### Shortest Path Problem & Solutions

If a graph contains no negative weight cycles reachable from the source vertex, then the shortest path is well defined, and may even have a negative value. 

we define the **shortest paht weight from $u$ to $v$** as $\delta(u, v)$

We define a nodes shortest path to a source as $v.d$

#### Optimal Substructure

* Let $p = < v_0, v_1,..., v_k>$ be the shortest path from $v_0$ to $v_k$
* For $0 \leq i \leq j \leq k$ be the subpath of $p_{ij}$ from $v_i$ to $v_j$
* Then $p_{ij}$ is also a shortest path

Many algorithms can be formalised as a repeated **relaxation**, testing whether the shortest path can be improved.

#### Properties

* **Triangle Inequality** - For an edge $(u, v)$, $\delta(S, v) \leq \delta(S, u) + w(u, v)$ 
* **Upper Bound** - once $v.d$ reaches $\delta$ it never changes
* **No path** - If there is no path, then $\delta = \inf$
* **Relaxation** - If $p = < v_0, v_1,..., v_k>$ is a shortest path, and we relax the edges in order, then the final vertex in p will have $v.d = \delta$ regardless of other steps
* **Predescessor subgraph** - once all $v.d = \delta$, the predecessor graph is a shortest path tree routed at $s$


#### Bellman-Ford

Progressive edge relaxation, decreasing an estimate for $v.d$ for each $v$ until the true $\delta$ is reached.

The algorithm will return true if and only if there are no reachable negative weight cycles

Runtime: $\Theta(VE)$

In [26]:
_V = ["s","a","b","c","d","e"]
ET= [("s", "a", 5), \
    ("a", "c", 5), \
    ("c", "d", 1), \
    ("d", "e", 4), \
    ("e", "b", 5), \
    ("b", "s", 2), \
    ("b", "a", 1), \
    ("a", "e", 2), \
    ("e", "c", 2)]

G = Graph(_V, None, None, None)
G.load_edge_from_tuple(ET)
G.to_undirected()

def bellman_ford(G: Graph, s: Vertex):
    for v in G.V:
        v.d = float('inf')
        v.pi = None
    s.d = 0

    for i in range(len(G.V)):
        # note that we have a separate edge instance for each direction 
        # so this works fine
        for e in G.E:
            a = G.get_vertex(e.a)
            b = G.get_vertex(e.b)
            # relax
            if b.d > a.d + e.w:
                b.d = a.d + e.w
                b.pi = a
    # check for negative acyclics i.e failures
    for e in G.E:
            a = G.get_vertex(e.a)
            b = G.get_vertex(e.b)
            if b.d > a.d + e.w:
                raise Exception
    return True

bellman_ford(G, G.get_vertex("s"))

for v in G.V:
    print(v)

s  C:[inf] d[0] f[0] [white]
a parent[b] C:[inf] d[3] f[0] [white]
b parent[s] C:[inf] d[2] f[0] [white]
c parent[e] C:[inf] d[7] f[0] [white]
d parent[c] C:[inf] d[8] f[0] [white]
e parent[a] C:[inf] d[5] f[0] [white]


#### Dijkstra

Maintains a ser of vertices $S$ whose final $v.d$ have been computed. Repeatedly select a non-final vertex with the lowest estimated $v.d$, adds it to $S$ and relaxes all edges that are connected to the vertex.

Use a min-priority queue for optimal next vertex selection.

Only functions with *non negative weights*

Runtime: $\Theta(|E| \log |V|)$ for a bin min-heap queue

In [57]:
import copy

_V = ["s","a","b","c","d","e"]
ET= [("s", "a", 5), \
    ("a", "c", 5), \
    ("c", "d", 1), \
    ("d", "e", 4), \
    ("e", "b", 5), \
    ("b", "s", 2), \
    ("b", "a", 1), \
    ("a", "e", 2), \
    ("e", "c", 2)]

G = Graph(_V, None, None, None)
G.load_edge_from_tuple(ET)
G.to_undirected()

def dijkstra(G: Graph, s: Vertex):
    for v in G.V:
        v.d = float('inf')
        v.pi = None
    s.d = 0

    S = []
    # deep copy vertex state
    Q = copy.deepcopy(G.V)
     
    while len(Q):
        # sort the vertices into ascending order and get min
        Q = sorted(Q, key=lambda x: x.d, reverse=False)
        u = Q.pop(0)
        S.append(u)

        # relax all edges attached to u
        for e in G.Aj[u.v]:
            w = G.Aj[u.v][e]
            a = G.get_vertex(u)
            b = G.get_vertex(e)
            # relax source
            if b.d > a.d + w:
                b.d = a.d + w
                b.pi = a
                # update locally too
                localb = Q[Q.index(b.v)]
                localb.d = a.d + w
                localb.pi = a

    return True

print( dijkstra(G, G.get_vertex("s")) )

for v in G.V:
    print(v)

True
s  C:[inf] d[0] f[0] [white]
a parent[b] C:[inf] d[3] f[0] [white]
b parent[s] C:[inf] d[2] f[0] [white]
c parent[e] C:[inf] d[7] f[0] [white]
d parent[c] C:[inf] d[8] f[0] [white]
e parent[a] C:[inf] d[5] f[0] [white]


#### Floyd-Warshall

Solves the **all pairs** shortest path using dynamic programming. Relies on the following statement:

The shortest path $D_{ij}$ from $i$ to $j$ via intermediate vertices ${1..k}$, is the minimum path of a) some $i \rightarrow k \rightarrow j$ and b) A shortest path from $i$ to $j$ that does not go through $k$, and uses only the intermediate vertices.

i.e. tail end pairs recursive relaxation

Is an A to B solver.

<img src="media/gfw.png" alt="drawing" width="800"/>


### Flow Networks and Maximum Flow

A flow network is a directed graph, where each edge as a strictly non-negative capacity. 

A flow network has two additional distinguished vertices, the *source* $s$ and the *sink* $t$. For simplicity we assume all vertices lie on a path from $s$ to $t$.

A **flow** $f(u, v)$ in a given flow network is a real valued function that satisfies the properties:
* $0 \leq f(u,v) \leq c(u, v)$ - never exceeds an edges capacity
* $\sum f(u, v) = \sum f(v, u)$ - Flow Conservation

The value of a flow equal its net sum leaving the source, $\sum f(s, v) - \sum f(v, s)$

<img src="media/flow.png" alt="drawing" width="600"/>



#### Max Flow Problem

For a given flow network with source $s$ and sink $t$, find a flow of maximum value.
* There may be multiple sources and sinks - these can be combined to create infinite capacity super source and sinks by linking them together.
* If there is an 'antiparallel edge', that being one where $(u, v) \& (v, u) \in \mathbb{E}$ split one edge into two with a central dummy vertex

#### Ford-Fulkerson Algorithm

* An edge of the flow can admit an amount of additional flow = capacity - existing flow
* This is called the *residual capacity* $c_f$
* The residual network is a graph of edges with capacities that represent how the flow can be changed
* An **augmenting path** is a simple path from $s$ to $t$ in the residual network
* The maximum amount by which we can increase the flow on an augmenting path is called its residual capacity as before

Method:
1. Initialise the flow to 0
2. while there is an augmenting path that exists in the residual network
3. augment the flow along that path

#### Edmonds-Karp

Choose the augmenting path as a shortest path from to in the residual network where
each edge has unit distance (weight).

#### Maximum Bipartite Matching

No idea....

